In [60]:
import numpy as np
from PIL import Image

In [61]:
input_img = Image.open('cameraman.tif').convert('L')

In [62]:
pixel_map = np.array(input_img)

In [63]:
gray_levels, gray_levels_counts = np.unique(pixel_map, return_counts=True)
indices = (-gray_levels_counts).argsort()
sorted_gray_levels = gray_levels[indices]

In [64]:
def reduce_source(pixel_probabilities):
    
    reduced_pixels = []
    summation_values_indices = []
    total_reductions = len(pixel_probabilities) - 2
    
    pixel_probabilitie = list(np.sort(pixel_probabilities)[::-1])
    first_reduction = list(pixel_probabilities[0:len(pixel_probabilities) - 2])    
    summed_value = np.sum(pixel_probabilities[total_reductions: len(pixel_probabilities)])
    first_reduction.append(summed_value)
    
    first_reduction.sort(reverse= True)
    reduced_pixels.append(first_reduction)
    
    summed_value_index = [index for index, value in enumerate(first_reduction) if value == summed_value]
    summation_values_indices.append(summed_value_index[len(summed_value_index)-1])
    
    for reduction_stage in range(total_reductions-1):
        
        reduction = []
        
        last_reduction_index = len(reduced_pixels) - 1
        last_reduction = list(reduced_pixels[last_reduction_index])
        last_reduction.sort(reverse= True)
        
        last_reduction_length = len(last_reduction)
        reduction.extend(last_reduction[0:len(last_reduction) - 2])
        summed_value = last_reduction[last_reduction_length-2 : last_reduction_length]
        summed_value = np.sum(summed_value)
        
        reduction.extend([summed_value])
        reduction.sort(reverse= True)
        reduced_pixels.append(reduction)
        
        summed_value_index = [index for index, value in enumerate(reduction) if value == summed_value]
        summation_values_indices.append(summed_value_index[len(summed_value_index)-1])
        
    return reduced_pixels, summation_values_indices
    

In [65]:
def encode_reductions(reductions, summed_values_indices):
    
    reduction_encodings = []
     
    reduction_encodings.append(['0','1'])
    
    sum_index_to_access = len(summed_values_indices) - 1
    sum_index = summed_values_indices[sum_index_to_access]
    
    reduction_encodings = encode_reductions_helper(reduction_encodings, sum_index)

    for encoding_stage in range(1,len(reductions)):
                
        index_to_access = len(summed_values_indices) - encoding_stage - 1
        sum_index = summed_values_indices[index_to_access]
        
        reduction_encodings = encode_reductions_helper(reduction_encodings, sum_index)   
           
    return reduction_encodings
               

In [66]:
def encode_reductions_helper(reduction_encodings, sum_index):
    
    encoding = []
    
    sum_encoding_list = reduction_encodings[0]
    sum_encoding = sum_encoding_list[sum_index]
    
    encoding.extend(sum_encoding_list[0:sum_index])
    encoding.extend(sum_encoding_list[sum_index+1: len(sum_encoding_list)])
    encoding.append(sum_encoding + '0')
    encoding.append(sum_encoding + '1')
    
    reduction_encodings.insert(0,encoding)
    
    return reduction_encodings

In [67]:
def encode_image(pixel_map, img_dictionary):
    
    img_binary_encoding = ''
    width, height = pixel_map.shape
    
    for width_index in range(0, width): 
        for height_index in range(0, height):
            
            pixel = pixel_map[width_index][height_index]
            pixel_encoding = img_dictionary.get(pixel)
            
            img_binary_encoding += pixel_encoding
        
    return img_binary_encoding

In [68]:
reductions, summation_indices = reduce_source(gray_levels_counts)
reduction_encodings = encode_reductions(reductions, summation_indices)

In [69]:
gray_levels = list(gray_levels)
gray_levels.sort(reverse = True)
dictionary = dict(zip(sorted_gray_levels, reduction_encodings[0]))

In [70]:
with open('Dict.txt', 'w') as file:
    for pixel_value, pixel_encoding in dictionary.items():
        file.write(str(pixel_value) + ' : '+ str(pixel_encoding) + '\n\n')

In [71]:
binary_encoding = encode_image(pixel_map, dictionary)

In [72]:
with open('BinCode.txt', 'w') as file:
        file.write(binary_encoding)